In [2]:

__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')


import openai
import os
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI


OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

# load document
loader = PyPDFLoader("../../example_paper1.pdf")
documents = loader.load()
print(len(documents))


text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
text = text_splitter.split_documents(documents)
print(len(text))

embeddings = OpenAIEmbeddings()
# embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
db = Chroma.from_documents(documents=text, 
                           embedding=embeddings)
print(db._collection.count())

retriever = db.as_retriever(search_type="similarity", search_kwargs={"k":2})

llm = ChatOpenAI(model_name="gpt-3.5-turbo-16k", temperature=0)
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff", 
    return_source_documents=True,
    # chain_type_kwargs=
)


question = "what does this paper talk about?"
result = qa_chain({"query": question})
print(result)

retriever.get_relevant_documents(question)


33
86
86
{'query': 'what does this paper talk about?', 'result': 'The paper discusses the evaluation of learning architectures for tabular data. It compares different benchmarking methodologies and highlights the importance of accounting for hyper-parameter choice. The paper also reveals clear trends, indicating that tree-based models tend to yield good predictions on tabular data with less computational effort.', 'source_documents': [Document(page_content='ImageNet: A large-scale hierarchical image database | IEEE Conference Publication | IEEE Xplore.\nhttps://ieeexplore.ieee.org/document/5206848.\nZachary C. Lipton and Jacob Steinhardt. Troubling Trends in Machine Learning Scholarship: Some\nML papers suffer from ﬂaws that could mislead the public and stymie future research. Queue, 17\n(1):Pages 80:45–Pages 80:77, February 2019. ISSN 1542-7730. doi: 10.1145/3317287.3328534.\nXavier Bouthillier, Pierre Delaunay, Mirko Bronzi, Assya Troﬁmov, Brennan Nichyporuk, Justin\nSzeto, Nazanin M

[Document(page_content='ImageNet: A large-scale hierarchical image database | IEEE Conference Publication | IEEE Xplore.\nhttps://ieeexplore.ieee.org/document/5206848.\nZachary C. Lipton and Jacob Steinhardt. Troubling Trends in Machine Learning Scholarship: Some\nML papers suffer from ﬂaws that could mislead the public and stymie future research. Queue, 17\n(1):Pages 80:45–Pages 80:77, February 2019. ISSN 1542-7730. doi: 10.1145/3317287.3328534.\nXavier Bouthillier, Pierre Delaunay, Mirko Bronzi, Assya Troﬁmov, Brennan Nichyporuk, Justin\nSzeto, Nazanin Mohammadi Sepahvand, Edward Raff, Kanika Madan, Vikram V oleti, Samira\nEbrahimi Kahou, Vincent Michalski, Tal Arbel, Chris Pal, Gael Varoquaux, and Pascal Vincent.\nAccounting for Variance in Machine Learning Benchmarks. Proceedings ofMachine Learning\nandSystems, 3:747–769, March 2021.\n9', metadata={'page': 8, 'source': '../../example_paper1.pdf'}),
 Document(page_content='methods, how would the evaluation change including missing d

In [4]:
print(result["result"])

The paper discusses the evaluation of learning architectures for tabular data. It compares different benchmarking methodologies and highlights the importance of accounting for hyper-parameter choice. The paper also reveals clear trends, indicating that tree-based models tend to yield good predictions on tabular data with less computational effort.


In [ ]:

__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

import openai
import os
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

OPENAI_API_KEY = os.environ["OPENAI_API_KEY"] 
# openai.api_key = os.environ.get("OPENAI_API_KEY")
# if not openai.api_key:
#     raise ValueError("Environment variable OPENAI_API_KEY not set")

# response = openai.ChatCompletion.create(
#     model = "gpt-4",
#     messages = [
#         {
#             "role": "user",
#             "content": "Summarise this page"
#         },
#     ]
# )
# page_summary = response["choices"][0]["message"]["content"]

# load document
# loader = PyPDFLoader("example_paper.pdf")
# documents = loader.load()
# print(len(documents))

#
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
# text = text_splitter.split_documents(documents)
# print(len(text))

# #
# embeddings = OpenAIEmbeddings()
# # embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# db = Chroma.from_documents(documents=text, 
#                            embedding=embeddings)
# llm = ChatOpenAI(model_name="gpt-3.5-turbo-16k", temperature=0)
# retriever = db.as_retriever(search_type="similarity", search_kwargs={"k":2})
# qa_chain = RetrievalQA.from_chain_type(
#     llm=llm,
#     retriever=retriever,
#     chain_type="stuff", 
#     return_source_documents=True,
#     # chain_type_kwargs=
# )


# question = "what does this paper talk about?"
# result = qa_chain({"query": question})
# print(result)

# 

from langchain.chains.summarize import load_summarize_chain
from langchain.prompts import PromptTemplate
from langchain.chains import MapReduceDocumentsChain, ReduceDocumentsChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains.llm import LLMChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import WebBaseLoader

# loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
# docs = loader.load()
# print(len(docs))

# load document
loader = PyPDFLoader("example_paper.pdf")
docs = loader.load()
print(len(docs))

## option 2
llm = ChatOpenAI(temperature=0)
chain = load_summarize_chain(llm, chain_type="map_reduce")
print(chain.run(docs))



# # define prompt
# # langchain doc: https://python.langchain.com/docs/use_cases/summarization
# llm = ChatOpenAI(temperature=0)
# # map
# map_template = """The following is a set of doucments
# {docs}
# Based on this list of docs, please identify the main themes
# Helpful Answer:"""
# map_prompt = PromptTemplate.from_template(map_template)
# map_chain = LLMChain(llm=llm, prompt=map_prompt)
# # reduce
# reduce_template = """The following is set of summaries:
# {doc_summaries}
# Take these and distill it into a final, consolidated summary of the main themes.
# Helpful Answer:"""
# reduce_prompt = PromptTemplate.from_template(reduce_template)
# reduce_chain = LLMChain(llm=llm, prompt=reduce_prompt)
 
# combine_documents_chain = StuffDocumentsChain(llm_chain=reduce_chain, document_variable_name="doc_summaries") # not sure what this is 
# reduce_documents_chain = ReduceDocumentsChain(
#     combine_documents_chain = combine_documents_chain,
#     collapse_documents_chain = combine_documents_chain,
#     token_max = 4000
# )
# # final chain
# map_reduce_chain = MapReduceDocumentsChain(
#     llm_chain = map_chain,
#     reduce_documents_chain = reduce_documents_chain,
#     document_variable_name = "docs",
#     return_intermediate_steps = False
# )

# text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
#     chunk_size=1000, chunk_overlap=0
# )
# split_docs = text_splitter.split_documents(docs)
# print(map_reduce_chain.run(split_docs))